In [ ]:
# %pip install transformers
# %pip install SentencePiece

In [ ]:
import os
import random
import pandas as pd
import numpy as np
import csv
import tensorflow as tf
import torch
from sklearn.model_selection import train_test_split
import textwrap
import progressbar as pgr
import keras
from tensorflow.keras.preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
import time
import datetime
import json

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
train_set = pd.read_csv("/content/drive/My Drive/BTP/train.csv")
test_set = pd.read_csv("/content/drive/My Drive/BTP/test.csv")
validation_set = pd.read_csv("/content/drive/My Drive/BTP/dev.csv")

In [ ]:
train_set.head()

In [ ]:
train_set['text'][0]

In [ ]:
print(train_set.shape)
print(test_set.shape)
print(validation_set.shape)

In [ ]:
import ast 

for i in range(42835):
  train_set['labels'][i]=ast.literal_eval(train_set['labels'][i])

for i in range(13039):
  test_set['labels'][i]=ast.literal_eval(test_set['labels'][i])

for i in range(10200):
  validation_set['labels'][i]=ast.literal_eval(validation_set['labels'][i])

type(train_set['labels'])

In [ ]:
type(train_set['labels'])

In [ ]:
validation_set['text'][0]

In [ ]:
train_set['labels'][0]

In [ ]:
from transformers import PreTrainedModel, PreTrainedTokenizer, PretrainedConfig
from transformers import BertForSequenceClassification, BertTokenizer, BertConfig
from transformers import RobertaForSequenceClassification, RobertaTokenizer, RobertaConfig
from transformers import XLNetForSequenceClassification, XLNetTokenizer, XLNetConfig
from transformers import XLMForSequenceClassification, XLMTokenizer, XLMConfig
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer, DistilBertConfig

In [ ]:
MODEL_CLASSES = {
    'bert': (BertForSequenceClassification, BertTokenizer, BertConfig),
    'xlnet': (XLNetForSequenceClassification, XLNetTokenizer, XLNetConfig),
    'xlm': (XLMForSequenceClassification, XLMTokenizer, XLMConfig),
    'roberta': (RobertaForSequenceClassification, RobertaTokenizer, RobertaConfig),
    'distilbert': (DistilBertForSequenceClassification, DistilBertTokenizer, DistilBertConfig)}

model_type = 'xlnet'
model_class, tokenizer_class, config_class = MODEL_CLASSES[model_type]
model_name = 'xlnet-base-cased'

tokenizer = tokenizer_class.from_pretrained(model_name)

In [ ]:
len(train_set['text'])

In [ ]:
print(train_set['labels'])
print(train_set.shape)

In [ ]:
# def input_id_maker(dataf, tokenizer):
#   input_ids = []
#   lengths = []
#   pgr.widgets = [pgr.Percentage(), ' ', pgr.Bar(), ' ', pgr.ETA()]
#   progress = pgr.ProgressBar(widgets=pgr.widgets, max_value=len(dataf['text']))

#   for i in progress(range(len(dataf['text']))):
#     sen = dataf['text'].iloc[i]
#     sen = tokenizer.tokenize(sen)
#     CLS = tokenizer.cls_token
#     SEP = tokenizer.sep_token
#     if(len(sen) > 510):
#       sen = sen[len(sen)-510:]

#     sen = sen + [SEP] + [CLS]
#     encoded_sent = tokenizer.convert_tokens_to_ids(sen)
#     input_ids.append(encoded_sent)
#     lengths.append(len(encoded_sent))

#   input_ids = pad_sequences(input_ids, maxlen=512, value=0, dtype="long", truncating="pre", padding="pre")
#   return input_ids, lengths

def input_id_maker(dataf, tokenizer):
  input_ids = []
  lengths = []
  temp=[0]
  pgr.widgets = [pgr.Percentage(), ' ', pgr.Bar(), ' ', pgr.ETA()]
  progress = pgr.ProgressBar(widgets=pgr.widgets, max_value=len(dataf['text']))
## missed the last 512 words extraction
  for i in progress(range(len(dataf['text']))):
    input_id_temp=[]
    sen = dataf['text'].iloc[i]
    sen = tokenizer.tokenize(sen)
    CLS = tokenizer.cls_token
    SEP = tokenizer.sep_token
    r_col=0
    l=len(sen)
    rg=0
    while(l > 510):
      t = sen[rg:rg+510] + [SEP]+ [CLS]
      encoded_sent = tokenizer.convert_tokens_to_ids(t)
      #temp[0]=encoded_sent
      input_id_temp.append(encoded_sent)
      l=l-510
      rg=rg+510
      r_col+=1
    if(l!=0):
      t=sen[rg: ]+ [SEP]+ [CLS]
      encoded_sent=[]
      encoded_sent.append(tokenizer.convert_tokens_to_ids(t))
      # encoded_sent=encoded_sent.reshape(1,encoded_sent.shape[0])
      encoded_sent = pad_sequences(encoded_sent, maxlen=512, value=0, dtype="long", truncating="pre", padding="pre")
      encoded_sent=encoded_sent.tolist()
      input_id_temp.append(encoded_sent[0])
      r_col+=1 #r_col contains count of chunks per record
    lengths.append(r_col)
    input_ids.append(input_id_temp)
  return input_ids, lengths

# First dataset is embeded using xlnet pretrained model. Later   these embedding dataset is used to finetune the xlnet.

In [ ]:
train_input_ids, train_lengths = input_id_maker(train_set, tokenizer)
validation_input_ids, validation_lengths = input_id_maker(validation_set, tokenizer)

In [ ]:
test_input_ids, test_lengths = input_id_maker(test_set, tokenizer)

In [ ]:
test_lengths[:100]

In [ ]:
import pickle

dir_path = '/content/drive/My Drive/BTP'

# Create an empty pickle file
file_path = dir_path + '/train_input.pkl'
with open(file_path, 'wb') as f:
    pass
file_path1 = dir_path + '/validation_input.pkl'
with open(file_path, 'wb') as f:
    pass
# Write your pickled object to the file
with open(file_path, 'wb') as f:
    pickle.dump(train_input_ids, f)

with open(file_path1, 'wb') as f:
    pickle.dump(validation_input_ids, f)

In [ ]:
length_path1 = dir_path + '/train_length.pkl'
length_path2 = dir_path + '/validation_length.pkl'

with open(length_path1, 'wb') as f:
    pickle.dump(train_lengths, f)

with open(length_path2, 'wb') as f:
    pickle.dump(validation_lengths, f)

In [ ]:
#execute
import pickle
len_path1= '/content/drive/My Drive/BTP/train_length.pkl'
with open(len_path1, 'rb') as f:
    # Use pickle.load to deserialize the object from the file
    train_lengths = pickle.load(f)
len_path2= '/content/drive/My Drive/BTP/validation_length.pkl'
with open(len_path2, 'rb') as f:
    # Use pickle.load to deserialize the object from the file
    validation_lengths = pickle.load(f)

max(train_lengths)

In [ ]:
#execute
import pickle
filepath1= '/content/drive/My Drive/BTP/train_input.pkl'
with open(filepath1, 'rb') as f:
    # Use pickle.load to deserialize the object from the file
    train_input_ids = pickle.load(f)
filepath2= '/content/drive/My Drive/BTP/validation_input.pkl'
with open(filepath2, 'rb') as f:
    # Use pickle.load to deserialize the object from the file
    validation_input_ids = pickle.load(f)

len(train_input_ids)

In [ ]:
print(len(train_input_ids[0][1]))
print(train_input_ids[0][0])

In [ ]:
print(validation_input_ids[0])

In [ ]:
# def att_masking(input_ids):
#   attention_masks = []
#   for sent in input_ids:
#     att_mask = [int(token_id > 0) for token_id in sent]
#     attention_masks.append(att_mask)
#   return attention_masks
# changed as per my model
def att_masking(input_ids):
  attention_masks = [] # it is also 1D array like input_ids[] having each element as list.
  for i in range(len(input_ids)):
    temp=[]
    for sent in input_ids[i]:     #here input_ids[i] contains list of embeded 512 tokens of record i. and sent points to each list in that.
      att_mask = [int(token_id > 0) for token_id in sent]
      temp.append(att_mask)
    attention_masks.append(temp)
  return attention_masks

In [ ]:
train_attention_masks = att_masking(train_input_ids)
validation_attention_masks = att_masking(validation_input_ids)

In [ ]:
test_attention_masks = att_masking(test_input_ids)

In [ ]:
test_attention_masks[0:2]

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
unique_sections_train = sorted(set(section for row in train_set['labels'] for section in row))
mlb_train = MultiLabelBinarizer(classes=unique_sections_train)
y_train0 = mlb_train.fit_transform(train_set['labels'])
# for i in range(train.shape[0]):
#     y_train0.append(train.loc[i,'labels'])  
    
unique_sections_dev = sorted(set(section for row in validation_set['labels'] for section in row))
mlb_dev = MultiLabelBinarizer(classes=unique_sections_dev)
y_dev0 = mlb_dev.fit_transform(validation_set['labels'])
# for i in range(dev.shape[0]):
#     y_dev0.append(dev.loc[i,'labels'])

unique_sections_test = sorted(set(section for row in test_set['labels'] for section in row))
mlb_test = MultiLabelBinarizer(classes=unique_sections_test)
y_test0 = mlb_test.fit_transform(test_set['labels'])
# for i in range(test.shape[0]):
#     y_test0.append(test.loc[i,'labels'])

print(mlb_train.classes_)
print(mlb_dev.classes_)
print(mlb_test.classes_)

train_labels = y_train0.astype('int')
validation_labels = y_dev0.astype('int')
test_labels = y_test0.astype('int')

y_train0[0]

In [ ]:
len(train_labels[0])

In [ ]:
temp1=[]
for i in range(len(train_input_ids)):
  temp1.append(len(train_input_ids[i]))

max(temp1)

In [ ]:
train_input_ids_arr = np.asarray(train_input_ids, dtype=object)
validation_input_ids_arr = np.asarray(validation_input_ids, dtype=object)
test_input_ids_arr = np.asarray(test_input_ids, dtype=object)
train_attention_masks_arr = np.asarray(train_attention_masks, dtype=object)
validation_attention_masks_arr = np.asarray(validation_attention_masks, dtype=object)
test_attention_masks_arr = np.asarray(test_attention_masks, dtype=object)
train_labels_arr = np.asarray(train_labels, dtype=object)
validation_labels_arr = np.asarray(validation_labels, dtype=object)
test_labels_arr = np.asarray(test_labels, dtype=object)

In [ ]:
train_input_ids_arr = np.array([np.array(x) for x in train_input_ids_arr)
validation_input_ids_arr = np.array([np.array(x) for x in validation_input_ids_arr])
test_input_ids_arr = np.array([np.array(x) for x in test_input_ids_arr])

In [ ]:
train_input_ids_path = "/content/drive/My Drive/BTP/input_ids/train_input_ids.npy"
validation_input_ids_path = "/content/drive/My Drive/BTP/input_ids/validation_input_ids.npy"
test_input_ids_path = "/content/drive/My Drive/BTP/input_ids/test_input_ids.npy"
train_attention_masks_path = "/content/drive/My Drive/BTP/masks/train_attention_masks.npy"
validation_attention_masks_path = "/content/drive/My Drive/BTP/masks/validation_attention_masks.npy"
test_attention_masks_path = "/content/drive/My Drive/BTP/masks/test_attention_masks.npy"
train_labels_path = "/content/drive/My Drive/BTP/labels/train_labels.npy"
validation_labels_path = "/content/drive/My Drive/BTP/labels/validation_labels.npy"
test_labels_path = "/content/drive/My Drive/BTP/labels/test_labels.npy"

np.save(train_input_ids_path, train_input_ids_arr)
np.save(validation_input_ids_path, validation_input_ids_arr)
np.save(test_input_ids_path, test_input_ids_arr)
np.save(train_attention_masks_path, train_attention_masks_arr)
np.save(validation_attention_masks_path, validation_attention_masks_arr)
np.save(test_attention_masks_path, test_attention_masks_arr)
np.save(train_labels_path, train_labels_arr)
np.save(validation_labels_path, validation_labels_arr)
np.save(test_labels_path, test_labels_arr)

In [ ]:
# train_inputs = train_input_ids
# validation_inputs = validation_input_ids
# train_masks = train_attention_masks
# validation_masks = validation_attention_mask
test = train_input_ids[1000:2000]
temp_tensor_list = [torch.tensor(lst) for lst in test]
max_len = 346
padded_tensors = []
for tensor in temp_tensor_list:
    # determine length of current tensor
    current_len = tensor.size(0)
    # if current tensor is shorter than max length, pad with zeros
    if current_len < max_len:
        pad_size = (max_len - current_len, tensor.size(1))
        tensor = torch.cat((tensor, torch.zeros(pad_size)), dim=0)
        tensor = tensor.type(torch.int64)
    # if current tensor is longer than max length, trim
    elif current_len > max_len:
        tensor = tensor.narrow(0, 0, max_len)
    padded_tensors.append(tensor)

# padded_tensors[0:5]
# import torch.nn.utils.rnn as rnn_utils
test_tensors = torch.stack(padded_tensors)
test_tensors.shape
# test_tensor1 = rnn_utils.pad_sequence([torch.tensor(lst) for lst in test], batch_first=True)
# train_inputs = torch.tensor(train_inputs)
# train_labels = torch.tensor(train_labels)
# train_masks = torch.stack([torch.stack([torch.tensor(item) for item in sublist]) for sublist in train_masks])
# train_masks = torch.tensor(train_masks)
# validation_inputs = torch.stack([torch.stack([torch.tensor(item) for item in sublist]) for sublist in validation_inputs])
# validation_inputs = torch.tensor(validation_inputs)
# validation_labels = torch.tensor(validation_labels)
# validation_masks = torch.tensor(validation_masks)
# validation_masks = torch.stack([torch.stack([torch.tensor(item) for item in sublist]) for sublist in validation_masks])
# # train_inputs = rnn_utils.pad_sequence([torch.tensor(lst) for lst in train_inputs], batch_first=True)
# # train_inputs = torch.tensor(train_inputs)
# train_labels = torch.tensor(train_labels)
# # train_masks = torch.stack([torch.stack([torch.tensor(item) for item in sublist]) for sublist in train_masks])
# # train_masks = torch.tensor(train_masks)
# # validation_inputs = torch.stack([torch.stack([torch.tensor(item) for item in sublist]) for sublist in validation_inputs])
# # validation_inputs = torch.tensor(validation_inputs)
# validation_labels = torch.tensor(validation_labels)
# # validation_masks = torch.tensor(validation_masks)
# # validation_masks = torch.stack([torch.stack([torch.tensor(item) for item in sublist]) for sublist in validation_masks])

In [ ]:
fold_dir = "/content/drive/My Drive/BTP/folds"
os.makedirs(fold_dir, exist_ok=True)
fold_filename = os.path.join(fold_dir, f"folds.pt")

with open(fold_filename, "ab") as f:
    torch.save(test_tensors, f)

In [ ]:
test_ten = torch.load("/content/drive/My Drive/BTP/folds/folds.pt")
test_ten.shape

In [ ]:
test_ten

In [ ]:
import math
import gc

fold_dir = "/content/drive/My Drive/BTP/folds"
os.makedirs(fold_dir, exist_ok=True)

max_fold_size = 2000
num_folds = math.ceil(len(train_input_ids) / max_fold_size)


for fold_num in range(num_folds):
    start_idx = fold_num * max_fold_size
    end_idx = min(start_idx + max_fold_size, len(train_input_ids))
    fold_input_ids = train_input_ids[start_idx:end_idx]
    
    temp_tensor_list = [torch.tensor(lst) for lst in fold_input_ids]
    max_len = 346
    for tensor in temp_tensor_list:
        padded_tensors = []
        # determine length of current tensor
        current_len = tensor.size(0)
        # if current tensor is shorter than max length, pad with zeros
        if current_len < max_len:
            pad_size = (max_len - current_len, tensor.size(1))
            tensor = torch.cat((tensor, torch.zeros(pad_size)), dim=0)
            tensor = tensor.type(torch.int64)
        # if current tensor is longer than max length, trim
        elif current_len > max_len:
            tensor = tensor.narrow(0, 0, max_len)
        padded_tensors.append(tensor)

    test_tensors = torch.stack(padded_tensors)
    fold_filename = os.path.join(fold_dir, f"fold_{fold_num}.pt")
    torch.save(test_tensors, fold_filename)
    
    # remove unnecessary data to free up memory
    del temp_tensor_list
    gc.collect()

In [ ]:
padded_tensors[0].shape

In [ ]:
train_input_temp = torch.tensor(padded_tensors)
train_input_temp.shape

In [ ]:
test_tensor1.shape

In [ ]:
test2 = train_inputs[5000:10001]
test_tensor2 = rnn_utils.pad_sequence([torch.tensor(lst) for lst in test], batch_first=True)

In [ ]:
train_inputs_tensor[0]

In [ ]:
batch_size = 600
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size = batch_size)
validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = RandomSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size = batch_size)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = XLNetForSequenceClassification.from_pretrained(model_name, num_labels=len(y_train0[0]))
model.to(device)

In [ ]:
lr = 2e-6
max_grad_norm = 1.0
epochs = 3
num_total_steps = len(train_dataloader)*epochs
num_warmup_steps = 1000
warmup_proportion = float(num_warmup_steps) / float(num_total_steps)  # 0.1
optimizer = AdamW(model.parameters(), lr=lr, correct_bias=False)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = num_warmup_steps, num_training_steps = num_total_steps)

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

seed_val = 21


np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
for step, batch in enumerate(train_dataloader):
    print("Train Batch Shape:", batch[0].shape, batch[1].shape, batch[2].shape)

for step, batch in enumerate(validation_dataloader):
    print("Validation Batch Shape:", batch[0].shape, batch[1].shape, batch[2].shape)


In [ ]:
loss_values = []

# For each epoch...
for epoch_i in range(0, epochs):
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    t0 = time.time()
    total_loss = 0

    model.train()

    for step, batch in enumerate(train_dataloader):
        if step % 40 == 0 and not step == 0:
            print('  Batch {:>5,}  of  {:>5,}. '.format(step, len(train_dataloader)))

        
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        model.zero_grad()        

        outputs = model(b_input_ids, attention_mask=b_input_mask, labels=b_labels)
        
        loss = outputs[0]
        total_loss += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()

    avg_train_loss = total_loss / len(train_dataloader)            
    loss_values.append(avg_train_loss)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
        
    print("")
    print("Running Validation...")

    t0 = time.time()

    model.eval()

    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    for batch in validation_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        
        with torch.no_grad():        
          outputs = model(b_input_ids, attention_mask=b_input_mask)
    
        logits = outputs[0]

        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy

        nb_eval_steps += 1
    # Report the final accuracy for this validation run.
    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))

print("")
print("Training complete!")

In [ ]:
output_dir = './XLNet_final/' # path to which the fine tuned model is to be saved

# Create output directory if needed
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("Saving model to %s" % output_dir)

# Save a trained model, configuration and tokenizer using `save_pretrained()`.
# They can then be reloaded using `from_pretrained()`
model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
model_to_save.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

# Good practice: save your training arguments together with the trained model
# torch.save(args, os.path.join(output_dir, 'training_args.bin'))

# Copy the model files to a directory in your Google Drive.
!cp -r ./XLNet_final/ "/content/Drive/My Drive/shared_by_vijit2/LNLP/XLNet/XLNet_right_model/"

In [ ]:
labels = test_set.label.to_numpy().astype(int)

input_ids, input_lengths = input_id_maker(test_set, tokenizer)
attention_masks = att_masking(input_ids)

# Convert to tensors.
prediction_inputs = torch.tensor(input_ids)
prediction_masks = torch.tensor(attention_masks)
prediction_labels = torch.tensor(labels)

# Set the batch size.  
batch_size = 6  

# Create the DataLoader.
prediction_data = TensorDataset(prediction_inputs, prediction_masks, prediction_labels)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)



In [ ]:
# Prediction on test set

print('Predicting labels for {:,} test sentences...'.format(len(prediction_inputs)))

# Put model in evaluation mode
model.eval()

# Tracking variables 
predictions , true_labels = [], []


# Predict 
for (step, batch) in enumerate(prediction_dataloader):
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, b_labels = batch
  

  
  # Telling the model not to compute or store gradients, saving memory and 
  # speeding up prediction
  with torch.no_grad():
      # Forward pass, calculate logit predictions
      outputs = model(b_input_ids, token_type_ids=None, 
                      attention_mask=b_input_mask)

  logits = outputs[0]

  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  
  # Store predictions and true labels
  predictions.append(logits)
  true_labels.append(label_ids)

print('    DONE.')

In [ ]:
predictions = np.concatenate(predictions, axis=0)
true_labels = np.concatenate(true_labels, axis=0)
pred_flat = np.argmax(predictions, axis=1).flatten()
labels_flat = true_labels.flatten()

flat_accuracy(predictions,true_labels)

In [ ]:
import os
import random
import pandas as pd
import numpy as np
import csv
import tensorflow as tf
import torch
from sklearn.model_selection import train_test_split
from google.colab import drive
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
import textwrap
import progressbar
import keras
from keras.preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
import time
import datetime
import random

In [ ]:
def metrics_calculator(preds, test_labels):
    cm = confusion_matrix(test_labels, preds)
    TP = []
    FP = []
    FN = []
    for i in range(0,2):
        summ = 0
        for j in range(0,2):
            if(i!=j):
                summ=summ+cm[i][j]

        FN.append(summ)
    for i in range(0,2):
        summ = 0
        for j in range(0,2):
            if(i!=j):
                summ=summ+cm[j][i]

        FP.append(summ)
    for i in range(0,2):
        TP.append(cm[i][i])
    precision = []
    recall = []
    for i in range(0,2):
        precision.append(TP[i]/(TP[i] + FP[i]))
        recall.append(TP[i]/(TP[i] + FN[i]))

    macro_precision = sum(precision)/2
    macro_recall = sum(recall)/2
    micro_precision = sum(TP)/(sum(TP) + sum(FP))
    micro_recall = sum(TP)/(sum(TP) + sum(FN))
    micro_f1 = (2*micro_precision*micro_recall)/(micro_precision + micro_recall)
    macro_f1 = (2*macro_precision*macro_recall)/(macro_precision + macro_recall)
    return macro_precision, macro_recall, macro_f1, micro_precision, micro_recall, micro_f1

macro_precision, macro_recall, macro_f1, micro_precision, micro_recall, micro_f1 = metrics_calculator(pred_flat, labels_flat)
print(macro_precision, macro_recall, macro_f1, micro_precision, micro_recall, micro_f1)

In [ ]:
# Set the batch size.  
batch_size = 6  

# Create the DataLoader.
prediction_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
prediction_sampler = SequentialSampler(validation_data)
prediction_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size = batch_size)

In [ ]:
# Prediction on test set

print('Predicting labels for {:,} test sentences...'.format(len(prediction_inputs)))

# Put model in evaluation mode
model.eval()

# Tracking variables 
predictions , true_labels = [], []


# Predict 
for (step, batch) in enumerate(prediction_dataloader):
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, b_labels = batch
  

  
  # Telling the model not to compute or store gradients, saving memory and 
  # speeding up prediction
  with torch.no_grad():
      # Forward pass, calculate logit predictions
      outputs = model(b_input_ids, token_type_ids=None, 
                      attention_mask=b_input_mask)

  logits = outputs[0]

  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  
  # Store predictions and true labels
  predictions.append(logits)
  true_labels.append(label_ids)

print('    DONE.')

In [ ]:
predictions = np.concatenate(predictions, axis=0)
true_labels = np.concatenate(true_labels, axis=0)
pred_flat = np.argmax(predictions, axis=1).flatten()
labels_flat = true_labels.flatten()

flat_accuracy(predictions,true_labels)

In [ ]:
macro_precision, macro_recall, macro_f1, micro_precision, micro_recall, micro_f1 = metrics_calculator(pred_flat, labels_flat)
print(macro_precision, macro_recall, macro_f1, micro_precision, micro_recall, micro_f1)